In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import log_loss, accuracy_score, classification_report, f1_score
import numpy as np

In [5]:
df  = pd.read_csv(r'DoS_Recon.csv')

In [6]:
df.columns = df.columns.str.replace(' ', '_')

In [7]:
df['label'] = df['label'].apply(lambda x: 0 if 'DoS' in str(x) else 1)  # Use -1 for unmatched labels if needed


In [8]:
df.label.unique()

array([0, 1])

In [9]:
X = df.drop(['label'],axis=1)
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y, train_size = 0.8, test_size = 0.2, shuffle=True,random_state = 0)

In [10]:
# Remove irrelevant features and select important features
def Feature_Importance_LGBM(data):
    features = data.drop(['label'],axis=1).values  # "label" should be changed to the target class variable name if different
    labels = data['label'].values

    # Extract feature names
    feature_names = list(data.drop(['label'],axis=1).columns)

    # Empty array for feature importances
    feature_importance_values = np.zeros(len(feature_names))
    model = lgb.LGBMRegressor(verbose = -1)
    model.fit(features, labels)
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': model.feature_importances_})

    # Sort features according to importance
    feature_importances = feature_importances.sort_values('importance', ascending = False).reset_index(drop = True)

    # Normalize the feature importances to add up to one
    feature_importances['normalized_importance'] = feature_importances['importance'] / feature_importances['importance'].sum()
    feature_importances['cumulative_importance'] = np.cumsum(feature_importances['normalized_importance'])

    cumulative_importance=0.90 # Only keep the important features with cumulative importance scores>=90%. It can be changed.

    # Make sure most important features are on top
    feature_importances = feature_importances.sort_values('cumulative_importance')

    # Identify the features not needed to reach the cumulative_importance
    record_low_importance = feature_importances[feature_importances['cumulative_importance'] > cumulative_importance]

    to_drop = list(record_low_importance['feature'])
    print(feature_importances.drop(['importance'],axis=1))
    return to_drop

In [11]:
# Remove redundant features
def Feature_Redundancy_Pearson(data):
    correlation_threshold=0.95 # Only remove features with the redundancy>90%. It can be changed
    features = data.drop(['label'],axis=1)
    corr_matrix = features.corr()

    # Extract the upper triangle of the correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(bool))

    # Select the features with correlations above the threshold
    # Need to use the absolute value
    to_drop = [column for column in upper.columns if any(upper[column].abs() > correlation_threshold)]

    # Dataframe to hold correlated pairs
    record_collinear = pd.DataFrame(columns = ['drop_feature', 'corr_feature', 'corr_value'])

    # Iterate through the columns to drop
    for column in to_drop:

        # Find the correlated features
        corr_features = list(upper.index[upper[column].abs() > correlation_threshold])

        # Find the correlated values
        corr_values = list(upper[column][upper[column].abs() > correlation_threshold])
        drop_features = [column for _ in range(len(corr_features))]

        # Record the information (need a temp df for now)
        temp_df = pd.DataFrame.from_dict({'drop_feature': drop_features,
                                         'corr_feature': corr_features,
                                         'corr_value': corr_values})
        record_collinear = pd.concat([record_collinear, temp_df], ignore_index = True)
#     print(record_collinear)
    return to_drop

In [12]:
def Auto_Feature_Engineering(s1):
    drop1 = Feature_Importance_LGBM(s1)
    df1 = s1.drop(columns = drop1)

    drop4 = Feature_Redundancy_Pearson(df1)
    df4 = df1.drop(columns = drop4)
    return df4

In [13]:
df1 = Auto_Feature_Engineering(df)

            feature  normalized_importance  cumulative_importance
0               IAT               0.198667               0.198667
1          Variance               0.102667               0.301333
2          Duration               0.087000               0.388333
3         syn_count               0.065333               0.453667
4         fin_count               0.053333               0.507000
5         rst_count               0.053333               0.560333
6           Tot_sum               0.045333               0.605667
7     Protocol_Type               0.045333               0.651000
8              Rate               0.043333               0.694333
9               Min               0.039667               0.734000
10    flow_duration               0.034333               0.768333
11    Header_Length               0.031667               0.800000
12             HTTP               0.030667               0.830667
13           Number               0.028667               0.859333
14        

C:\Users\Dell\AppData\Local\Temp\ipykernel_17848\3633410587.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  record_collinear = pd.concat([record_collinear, temp_df], ignore_index = True)
